# S09 Lab Exercise 

## Víctor Vega Sobral

### Explanations 
The attached files are a collection of tweets labelled with sentiment in 3 categories:

```json
sentiments = {
    "LABEL_0": "Bearish", 
    "LABEL_1": "Bullish", 
    "LABEL_2": "Neutral"
}  
```

1. Train a LSTM network to with the training file. 

2. Validate the trained model with the valid file. 

3. Comment what you are doing in each part of your code. As the better the code, comments and result validation as the better the grade.

Remember that you have to send the final file in this exercise and the file must be in your digital portfolio with all the proper commits done.

- ``sent_train.csv`` 
- ``sent_valid.csv`` 



In [ ]:
__author__ = "Victor Vega Sobral"

---

### 1. Importing necessary Libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.metrics import Precision, Recall

---

### 2. Loading the Datasets

Next step is to load the two different datasets in:

- `train_df`: Dataframe with the training set.

- `valid_df`: Dataframe with the validation set.

---

#### 2.1 Train dataset

In [ ]:
# Training dataset
train_df = pd.read_csv("data/sent_train.csv")
train_df.info()

X_train = train_df["text"]
y_train = train_df["label"]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9543 entries, 0 to 9542
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    9543 non-null   object
 1   label   9543 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 149.2+ KB


#### 2.4 Test Dataset

In [ ]:
# Valdiation dataset
valid_df = pd.read_csv("data/sent_valid.csv")
valid_df.info()

X_test = valid_df["text"]
y_test = valid_df["label"]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2388 entries, 0 to 2387
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2388 non-null   object
 1   label   2388 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 37.4+ KB


---

### 3. Dividing in train and test split (explanation)

As we already have two different csv files with the train and validation, this step is already done. In other cases, `train_test_split` scikit-learn method could be used.

---

### 4. Tokenization and Paddding

Before training the LSTM, we need to convert both test and training dataframes to a sequence of numbers using the Keras tokenizer.

* `num_words`: defines the maximum number of words that the LSTM will take into account.
* `max_len`: maximum length of each sequence. This is the *paddding* step. 
* `embedding_dim`: maximum dimmensions the embedding vector will have.

In [ ]:
############ TOKENIZATION ############
max_words = 10000
max_len = 100
embedding_dim = 64 # Number of dimensions the embedding vectors will have

# Instanciating and adjusting tokenizator
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts()

# Converting texts to numeric sequences.
X_train_seq = tokenizer.texts_to_sequences()
X_test_seq = tokenizer.texts_to_sequences()